In [14]:
# Apply improvements: rename x_flag to control_flag, shorten docstrings, unify commenting

def ascii_to_bits(character):
    """Convert ASCII character to 8-bit binary."""
    return f"{ord(character):08b}"

def bits_to_ascii(bits):
    """Convert 8-bit binary to ASCII character."""
    return chr(int(bits, 2))

def calculate_parity(bits):
    """Return parity bit to ensure odd parity."""
    return '0' if sum(int(b) for b in bits) % 2 == 1 else '1'

def build_data_char(ascii_char, prev_payload):
    """Build data character (P + 0 + 8-bit payload)."""
    control_flag = '0'
    payload = ascii_to_bits(ascii_char)
    p = calculate_parity(prev_payload + control_flag)
    return p, control_flag, payload

def build_control_char(name, prev_payload):
    """Build control character (P + 1 + 2-bit control code)."""
    codes = {
        "FCT": "00",
        "EOP": "01",
        "EEP": "10",
        "Escape": "11"
    }
    if name not in codes:
        raise ValueError(f"Invalid control character name: {name}")
    control_flag = '1'
    payload = codes[name]
    p = calculate_parity(prev_payload + control_flag)
    return p, control_flag, payload

def create_null(prev_payload):
    """Build a NULL character (P + 1 + 110100)."""
    control_flag = '1'
    payload = '110100'
    p = calculate_parity(prev_payload + control_flag)
    return p + control_flag + payload

def create_fct(prev_payload):
    """Build an FCT character."""
    p, control_flag, payload = create_character("FCT", is_control=True, prev_payload=prev_payload)
    return p + control_flag + payload

def create_character(char, is_control, prev_payload):
    """Unified builder for data/control characters."""
    return build_control_char(char, prev_payload) if is_control else build_data_char(char, prev_payload)

def create_data_packet(data, address, initial_prev_payload):
    """Builds a SpaceWire data packet (addr + payload + EOP)."""
    packet = ""
    prev_payload = initial_prev_payload

    for char in address:
        p, control_flag, payload = build_data_char(char, prev_payload)
        packet += p + control_flag + payload
        prev_payload = payload

    for char in data:
        p, control_flag, payload = build_data_char(char, prev_payload)
        packet += p + control_flag + payload
        prev_payload = payload

    p, control_flag, payload = build_control_char("EOP", prev_payload)
    packet += p + control_flag + payload

    return packet

def create_clock(length):
    """Generate clock sequence starting at 1 (e.g., 1010...)."""
    return ''.join(str((i + 1) % 2) for i in range(length))

def create_strobe(bits, clock):
    """Generate strobe signal by XORing data and clock."""
    return ''.join(str(int(b) ^ int(c)) for b, c in zip(bits, clock))

def create_reset_signal(signal_length, reset_position):
    """Generate reset line with single 1 at reset_position."""
    reset = ['0'] * signal_length
    if 0 <= reset_position < signal_length:
        reset[reset_position] = '1'
    return ''.join(reset)

def save_to_file(data, strobe, reset, filename):
    """Save data, strobe, and reset signals to file."""
    with open(filename, "w") as file:
        file.writelines(f"{d}\t{s}\t{r}\n" for d, s, r in zip(data, strobe, reset))

def generate_transmission(null_count, include_fct, payload, buffer_length=1, initial_prev_payload=""):
    """Generate complete SpaceWire transmission."""
    bitstream = ""
    prev_payload = initial_prev_payload

    for _ in range(null_count):
        null_char = create_null(prev_payload)
        bitstream += null_char
        prev_payload = "110100"

    if include_fct:
        p, control_flag, payload_bits = create_character("FCT", is_control=True, prev_payload=prev_payload)
        bitstream += p + control_flag + payload_bits
        prev_payload = payload_bits

    packet = create_data_packet(payload, address="", initial_prev_payload=prev_payload)
    bitstream = bitstream + packet

    clk = create_clock(len(bitstream))
    strobe = create_strobe(bitstream, clk)
    reset = create_reset_signal(len(bitstream), reset_position=0)
    
    # Create buffer if specified
    if buffer_length > 0:
        buffer = '0' * buffer_length
        bitstream = buffer + bitstream
        strobe = buffer + strobe
        reset = buffer + reset

    return bitstream, strobe, reset


In [15]:
# Run the transmission generation
bitstream, strobe, reset = generate_transmission(
    null_count=11,
    include_fct=True,
    payload="H",
    buffer_length=1,
    initial_prev_payload=""
)

# Preview output
print(bitstream)
print(strobe)
print(reset)

# Save to file
save_to_file(bitstream, strobe, reset, "transmission.txt")

00111010011110100111101001111010011110100111101001111010011110100111101001111010011110100110010010010000101
01101111001011110010111100101111001011110010111100101111001011110010111100101111001011110011000111000101111
01000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
